# Anomaly Classification

## Import Everything

In [1]:
import importlib
import os
import shutil
from typing import Any

from otx.algorithms.anomaly.adapters.anomalib.data.dataset import (
    AnomalyClassificationDataset,
)
from otx.api.configuration.helper import create as create_hyper_parameters
from otx.api.entities.datasets import DatasetEntity
from otx.api.entities.inference_parameters import InferenceParameters
from otx.api.entities.label_schema import LabelSchemaEntity
from otx.api.entities.model import ModelEntity
from otx.api.entities.model_template import (
    ModelTemplate,
    TaskType,
    parse_model_template,
)
from otx.api.entities.optimization_parameters import OptimizationParameters
from otx.api.entities.resultset import ResultSetEntity
from otx.api.entities.subset import Subset
from otx.api.entities.task_environment import TaskEnvironment
from otx.api.entities.train_parameters import TrainParameters
from otx.api.usecases.adapters.model_adapter import ModelAdapter
from otx.api.usecases.tasks.interfaces.export_interface import ExportType
from otx.api.usecases.tasks.interfaces.optimization_interface import OptimizationType


## Setup Subsets

In [2]:
train_subset = {
    "ann_file": "../../data/anomaly/classification/train.json",
    "data_root": "../../data/anomaly/shapes",
}

val_subset = {"ann_file": "../../data/anomaly/classification/val.json", "data_root": "../../data/anomaly/shapes"}

test_subset = {
    "ann_file": "../../data/anomaly/classification/test.json",
    "data_root": "../../data/anomaly/shapes",
}


## Create Task

### Load Model Template

In [3]:
model_template_path = "../../otx/algorithms/anomaly/configs/classification/padim/template.yaml"

model_template = parse_model_template(model_template_path)
model_template.task_type


ANOMALY_CLASSIFICATION

### Create Dataset

In [4]:
dataset = AnomalyClassificationDataset(train_subset=train_subset, val_subset=val_subset, test_subset=test_subset)
dataset.get_labels()


[LabelEntity(1, name=Anomalous, hotkey=, domain=ANOMALY_CLASSIFICATION, color=Color(red=75, green=168, blue=234, alpha=255), is_anomalous=True),
 LabelEntity(0, name=Normal, hotkey=, domain=ANOMALY_CLASSIFICATION, color=Color(red=71, green=200, blue=25, alpha=255), is_anomalous=False)]

### 

### Create Task and Task Environment

In [5]:
def create_task_environment(model_template: ModelTemplate, dataset: DatasetEntity) -> TaskEnvironment:
    """Create task environment."""
    hyper_parameters = create_hyper_parameters(model_template.hyper_parameters.data)
    labels = dataset.get_labels()
    label_schema = LabelSchemaEntity.from_labels(labels)

    return TaskEnvironment(
        model_template=model_template,
        model=None,
        hyper_parameters=hyper_parameters,
        label_schema=label_schema,
    )


In [6]:
def create_task(model_template: ModelTemplate, task_environment: TaskEnvironment, task: str) -> Any:
    """Create base torch or openvino task.

    Args:
        task (str): task type. Either base or openvino.

    Returns:
        Any: Base Torch or OpenVINO Task Class.

    Example:
        >>> create_task(model_template=model_template, task="base")
        <anomaly_classification.torch_task.AnomalyClassificationTask>

    """
    if model_template.entrypoints is not None:
        task_path = getattr(model_template.entrypoints, task)
    else:
        raise ValueError(f"Cannot create {task} task. `model_template.entrypoint` does not have {task}")

    module_name, class_name = task_path.rsplit(".", 1)
    module = importlib.import_module(module_name)
    return getattr(module, class_name)(task_environment=task_environment)


In [7]:
task_environment = create_task_environment(model_template, dataset)
task_environment


TaskEnvironment(model=None, label_schema=LabelSchemaEntity(label_groups=[LabelGroup(id=6331ac071e334d2562411e60, name=from_label_list, group_type=LabelGroupType.EXCLUSIVE, labels=[LabelEntity(0, name=Normal, hotkey=, domain=ANOMALY_CLASSIFICATION, color=Color(red=71, green=200, blue=25, alpha=255), is_anomalous=False), LabelEntity(1, name=Anomalous, hotkey=, domain=ANOMALY_CLASSIFICATION, color=Color(red=75, green=168, blue=234, alpha=255), is_anomalous=True)])]), hyper_params=CONFIGURABLE_PARAMETERS(header='Configuration for Padim', description='Configuration for Padim', visible_in_ui=True, id=ID()))

In [8]:
torch_task = create_task(model_template, task_environment, "base")
torch_task


/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/nncf/torch/dynamic_graph/patch_pytorch.py:163: UserWarning: Not patching unique_dim since it is missing in this version of PyTorch
  warnings.warn("Not patching {} since it is missing in this version of PyTorch".format(op_name))
/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/defusedxml/__init__.py:30: DeprecationWarning: defusedxml.cElementTree is deprecated, import from defusedxml.ElementTree instead.
  from . import cElementTree
/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: ote-alpha is an invalid version and will not be supported in a future release
  warnings.warn(
/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/pkg_resources/_vendor/packaging/version.py:111: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(
[INFO] 2022-09-26 15:41:29,445 - o

### Train

In [9]:
torch_output_model = ModelEntity(train_dataset=dataset, configuration=task_environment.get_model_configuration())
torch_task.train(dataset, torch_output_model, TrainParameters())


[INFO] 2022-09-26 15:41:29,905 - otx.algorithms.anomaly.tasks.train - Training the model.
[INFO] 2022-09-26 15:41:29,938 - otx.algorithms.anomaly.tasks.train - Training Configs '{'dataset': {'name': 'mvtec', 'format': 'mvtec', 'path': './datasets/MVTec', 'category': 'bottle', 'task': 'classification', 'image_size': [256, 256], 'train_batch_size': 32, 'test_batch_size': 32, 'num_workers': 8, 'transform_config': {'train': None, 'val': None}, 'create_validation_set': False, 'tiling': {'apply': False, 'tile_size': None, 'stride': None, 'remove_border_count': 0, 'use_random_tiling': False, 'random_tile_count': 16}}, 'model': {'name': 'padim', 'backbone': 'resnet18', 'pre_trained': True, 'layers': ['layer1', 'layer2', 'layer3'], 'normalization_method': 'min_max', 'input_size': [256, 256]}, 'metrics': {'image': ['F1Score', 'AUROC'], 'pixel': ['F1Score', 'AUROC'], 'threshold': {'image_default': 3, 'pixel_default': 3, 'adaptive': True}}, 'visualization': {'show_images': False, 'save_images': Tr

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[DEBUG-HPO] logged_metrics = {'image_F1Score': tensor(0.8889, device='cuda:0'), 'image_AUROC': tensor(0.5625, device='cuda:0')}


[INFO] 2022-09-26 15:41:33,703 - otx.algorithms.anomaly.tasks.inference - Saving the model weights.
[INFO] 2022-09-26 15:41:33,857 - otx.algorithms.anomaly.tasks.train - Training completed.


### Get Performance of Torch Model

In [10]:
def print_performance(task: Any, output_model: ModelEntity, dataset: DatasetEntity):
    ground_truth_validation_dataset = dataset.get_subset(Subset.VALIDATION)
    prediction_validation_dataset = task.infer(
        dataset=ground_truth_validation_dataset.with_empty_annotations(),
        inference_parameters=InferenceParameters(is_evaluation=True),
    )

    result_set = ResultSetEntity(
        model=output_model,
        ground_truth_dataset=ground_truth_validation_dataset,
        prediction_dataset=prediction_validation_dataset,
    )

    task.evaluate(result_set)
    print(result_set.performance)


In [11]:
print_performance(torch_task, torch_output_model, dataset)


[INFO] 2022-09-26 15:41:34,290 - otx.algorithms.anomaly.tasks.inference - Performing inference on the validation set using the base torch model.
[INFO] 2022-09-26 15:41:34,303 - otx.algorithms.anomaly.tasks.inference - Inference Configs '{'dataset': {'name': 'mvtec', 'format': 'mvtec', 'path': './datasets/MVTec', 'category': 'bottle', 'task': 'classification', 'image_size': [256, 256], 'train_batch_size': 32, 'test_batch_size': 32, 'num_workers': 8, 'transform_config': {'train': None, 'val': None}, 'create_validation_set': False, 'tiling': {'apply': False, 'tile_size': None, 'stride': None, 'remove_border_count': 0, 'use_random_tiling': False, 'random_tile_count': 16}}, 'model': {'name': 'padim', 'backbone': 'resnet18', 'pre_trained': True, 'layers': ['layer1', 'layer2', 'layer3'], 'normalization_method': 'min_max', 'input_size': [256, 256]}, 'metrics': {'image': ['F1Score', 'AUROC'], 'pixel': ['F1Score', 'AUROC'], 'threshold': {'image_default': 3, 'pixel_default': 3, 'adaptive': True}

Predicting: 0it [00:00, ?it/s]

[INFO] 2022-09-26 15:41:34,623 - otx.algorithms.anomaly.adapters.anomalib.callbacks.inference - 
	Threshold: 34.800, Assigned Label 'Anomalous', 0.626 min: 0.4732818603515625 max: 49.69810104370117
[INFO] 2022-09-26 15:41:34,624 - otx.algorithms.anomaly.adapters.anomalib.callbacks.inference - 
	Threshold: 34.800, Assigned Label 'Normal', 0.457 min: 0.4732818603515625 max: 49.69810104370117
[INFO] 2022-09-26 15:41:34,625 - otx.algorithms.anomaly.adapters.anomalib.callbacks.inference - 
	Threshold: 34.800, Assigned Label 'Anomalous', 0.803 min: 0.4732818603515625 max: 49.69810104370117
[INFO] 2022-09-26 15:41:34,626 - otx.algorithms.anomaly.adapters.anomalib.callbacks.inference - 
	Threshold: 34.800, Assigned Label 'Normal', 0.495 min: 0.4732818603515625 max: 49.69810104370117
[INFO] 2022-09-26 15:41:34,627 - otx.algorithms.anomaly.adapters.anomalib.callbacks.inference - 
	Threshold: 34.800, Assigned Label 'Anomalous', 0.500 min: 0.4732818603515625 max: 49.69810104370117
[INFO] 2022-09-2

Performance(score: 0.8333333333333333, dashboard: (4 metric groups))


### Export to OpenVINO

In [12]:
exported_model = ModelEntity(
    train_dataset=dataset,
    configuration=task_environment.get_model_configuration(),
)
torch_task.export(ExportType.OPENVINO, exported_model)
task_environment.model = exported_model


[INFO] 2022-09-26 15:41:34,866 - otx.algorithms.anomaly.tasks.inference - Exporting the OpenVINO model.
/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: ote-alpha is an invalid version and will not be supported in a future release
  warnings.warn(
/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: ote-alpha is an invalid version and will not be supported in a future release
  warnings.warn(


Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/tmp/otx-anomaliblnf_wqin/onnx_model.onnx
	- Path for generated IR: 	/tmp/otx-anomaliblnf_wqin
	- IR output name: 	onnx_model
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of legacy Frontend of Model Optimizer for model conversion into IR: 	False
	- Force the usage of new 

/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: ote-alpha is an invalid version and will not be supported in a future release
  warnings.warn(


OpenVINO runtime found in: 	/home/ashwin/miniconda3/envs/ote/lib/python3.8/site-packages/openvino
OpenVINO runtime version: 	2022.1.0-7019-cdb9bec7210-releases/2022/1
Model Optimizer version: 	2022.1.0-7019-cdb9bec7210-releases/2022/1
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /tmp/otx-anomaliblnf_wqin/onnx_model.xml
[ SUCCESS ] BIN file: /tmp/otx-anomaliblnf_wqin/onnx_model.bin
[ SUCCESS ] Total execution time: 0.65 seconds. 
[ SUCCESS ] Memory consumed: 559 MB. 
It's been a while, check for a new version of Intel(R) Distribution of OpenVINO(TM) toolkit here https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/download.html?cid=other&source=prod&campid=ww_2022_bu_IOTG_OpenVINO-2022-1&content=upg_all&medium=organic or on the GitHub*
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, p

### Get Performance of OpenVINO Model

In [13]:
openvino_task = create_task(model_template, task_environment, task="openvino")
print_performance(openvino_task, exported_model, dataset)


[INFO] 2022-09-26 15:41:37,496 - otx.algorithms.anomaly.tasks.openvino - Initializing the OpenVINO task.
[INFO] 2022-09-26 15:41:37,712 - otx.algorithms.anomaly.tasks.openvino - Start OpenVINO inference.


Performance(score: 0.8333333333333333, dashboard: (1 metric groups))


### POT Optimization

In [14]:
optimized_model = ModelEntity(
    dataset,
    configuration=task_environment.get_model_configuration(),
)
openvino_task.optimize(
    optimization_type=OptimizationType.POT,
    dataset=dataset.get_subset(Subset.TRAINING),
    output_model=optimized_model,
    optimization_parameters=OptimizationParameters(),
)
task_environment.model = optimized_model


[INFO] 2022-09-26 15:41:38,118 - otx.algorithms.anomaly.tasks.openvino - Starting POT optimization.
[INFO] 2022-09-26 15:42:02,266 - otx.algorithms.anomaly.tasks.openvino - POT optimization completed


### Get Performance of POT Optimized Model

In [15]:
print_performance(openvino_task, optimized_model, dataset)


[INFO] 2022-09-26 15:42:02,479 - otx.algorithms.anomaly.tasks.openvino - Start OpenVINO inference.


Performance(score: 0.3333333333333332, dashboard: (1 metric groups))


### NNCF Optimization

In [16]:
# Initialize model from weights.
init_model = ModelEntity(
    dataset,
    configuration=task_environment.get_model_configuration(),
    model_adapters={"weights.pth": ModelAdapter(torch_output_model.get_data("weights.pth"))},
)

task_environment.model = init_model

nncf_task = create_task(model_template, task_environment, task="nncf")
optimized_model = ModelEntity(
    dataset,
    configuration=task_environment.get_model_configuration(),
)
nncf_task.optimize(OptimizationType.NNCF, dataset, optimized_model)
task_environment.model = optimized_model


[INFO] 2022-09-26 15:42:02,928 - otx.algorithms.anomaly.tasks.inference - Initializing the task environment.
[INFO] 2022-09-26 15:42:03,281 - otx.algorithms.anomaly.tasks.nncf - Loaded model weights from Task Environment
[INFO] 2022-09-26 15:42:03,286 - otx.algorithms.anomaly.tasks.nncf - Optimization the model.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_predict_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
[INFO] 2022-09-26 15:42:03,298 - otx.algorithms.anomaly.adapters.anomalib

INFO:nncf:Please, provide execution parameters for optimal model initialization
INFO:nncf:Please, provide execution parameters for optimal model initialization
INFO:nncf:Wrapping module PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_extractor]/Conv2d[conv1] by PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_extractor]/NNCFConv2d[conv1]
INFO:nncf:Wrapping module PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_extractor]/BatchNorm2d[bn1] by PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_extractor]/NNCFBatchNorm2d[bn1]
INFO:nncf:Wrapping module PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_extractor]/Sequential[layer1]/BasicBlock[0]/Conv2d[conv1] by PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_extractor]/Sequential[layer1]/BasicBlock[0]/NNCFConv2d[conv1]
INFO:nncf:Wrapping module PadimModel/FeatureExtractor[feature_extractor]/FeatureListNet[feature_ext

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | image_threshold       | AdaptiveThreshold        | 0     
1 | pixel_threshold       | AdaptiveThreshold        | 0     
2 | model                 | NNCFNetwork              | 2.8 M 
3 | normalization_metrics | MinMax                   | 0     
4 | image_metrics         | AnomalibMetricCollection | 0     
5 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
2.8 M     Trainable params
57        Non-trainable params
2.8 M     Total params
11.141    Total estimated model params size (MB)
[INFO] 2022-09-26 15:42:06,189 - otx.algorithms.anomaly.adapters.anomalib.data.data - Global annotations: 12
[INFO] 2022-09-26 15:42:06,190 - otx.algorithms.anomaly.adapters.anomalib.data.data - Local annotations: 4
[INFO] 2022-09-26 15:42:06,190 - otx.algo

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

[INFO] 2022-09-26 15:42:08,233 - otx.algorithms.anomaly.tasks.inference - Saving the model weights.
[INFO] 2022-09-26 15:42:08,534 - otx.algorithms.anomaly.tasks.nncf - Training completed.


### Get Performance on NNCF Optimized Torch Model

In [17]:
print_performance(openvino_task, exported_model, dataset)


[INFO] 2022-09-26 15:42:08,765 - otx.algorithms.anomaly.tasks.openvino - Start OpenVINO inference.


Performance(score: 0.3333333333333332, dashboard: (1 metric groups))


## Clean-Up

In [18]:
results_dir = "./results"
if os.path.exists(results_dir):
    shutil.rmtree(results_dir)
